In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import wget
from pyspark.ml.feature import Bucketizer,RegexTokenizer,StopWordsRemover,CountVectorizer,IDF
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline,PipelineModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator

1. Dữ liệu Huấn luyện
Mô hình được xây dựng dựa trên một tập dữ liệu lớn chứa các đánh giá sản phẩm đã được gán nhãn là tích cực hoặc tiêu cực. Tập dữ liệu này cung cấp các ví dụ để mô hình học cách phân loại cảm xúc từ văn bản.
2. Quá trình Huấn luyện
Trong quá trình huấn luyện, mô hình sử dụng các thuật toán học máy để tìm ra các mẫu và mối quan hệ giữa các từ trong văn bản và nhãn cảm xúc tương ứng. Các thuật toán này có thể bao gồm Logistic Regression, Naive Bayes, hoặc các mô hình phức tạp hơn như mạng nơ-ron.
Mô hình học cách xác định các từ hoặc cụm từ có thể chỉ ra cảm xúc tích cực hoặc tiêu cực. Ví dụ, từ "tuyệt vời" có thể được gán nhãn tích cực, trong khi "thất vọng" có thể được gán nhãn tiêu cực.
3. Tính năng Văn bản
Mô hình chuyển đổi văn bản thành các vector số thông qua các phương pháp như Bag of Words hoặc TF-IDF. Điều này giúp mô hình hiểu được cấu trúc và ngữ nghĩa của văn bản.
Các tính năng này cho phép mô hình phân tích các yếu tố ngữ nghĩa trong văn bản, từ đó đưa ra quyết định về cảm xúc.
4. Dự đoán cho Dữ liệu Mới
Khi mô hình đã được huấn luyện, nó có thể được áp dụng cho các dữ liệu mới mà không cần gán nhãn. Mô hình sẽ nhận đầu vào là văn bản chưa được gán nhãn và sử dụng các kiến thức đã học để dự đoán cảm xúc.
Mô hình sẽ trả về xác suất cho từng lớp cảm xúc (tích cực hoặc tiêu cực), từ đó quyết định lớp nào mà văn bản thuộc về dựa trên một ngưỡng xác định (thường là 0.5).
5. Khả năng Tái sử dụng
Mô hình có thể được áp dụng cho nhiều loại văn bản khác nhau, miễn là chúng có ngữ cảnh tương tự với dữ liệu huấn luyện. Ví dụ, mô hình được huấn luyện trên các đánh giá sản phẩm có thể cũng hoạt động tốt với các bình luận trên mạng xã hội hoặc phản hồi của khách hàng.

<span style="color: blue; font-size:20px; font-weight:bold ">Tạo SparkSession và cấu hình để tương tác với Kafka và MongoDB</span> 

In [2]:
#Spark Session creation configured to interact with Kfka and MongoDB
spark = SparkSession.builder.appName("pyspark-notebook").\
config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-avro_2.12:3.0.0,org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
config("spark.mongodb.input.uri","mongodb://intent-mongo-1:27017/twitter_db.tweets").\
config("spark.mongodb.output.uri","mongodb://intent-mongo-1:27017/twitter_db.tweets").\
getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-254c78c2-cd8c-4e8b-b7d7-2995a46e7a42;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in

In [3]:
# spark.read.json("reviews_Sports_and_Outdoors_5.json.gz").show(35)

<span style="color: blue; font-size:20px; font-weight:bold ">Tải dữ liệu nếu không tồn tại và đọc nó vào Spark DataFrame</span> 

In [4]:
#Download dataset if not exists and read it as spark dataframe
try:
    df0 = spark.read.json("reviews_Sports_and_Outdoors_5.json.gz")
except Exception as e:
    url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz"
    wget.download(url)
    df0 = spark.read.json("reviews_Sports_and_Outdoors_5.json.gz")

df = df0.withColumn("text",concat(col("summary"), lit(" "),col("reviewText")))\
 .drop("helpful")\
 .drop("reviewerID")\
 .drop("reviewerName")\
 .drop("reviewTime")
df.count()

296337

In [5]:
print(spark.version)

3.0.0


<span style="color: green; font-size:30px; font-weight:bold ">Mô hình hóa và phân tích dữ liệu</span> 

<span style="color: blue; font-size:20px; font-weight:bold ">Mô tả dữ liệu đánh giá</span> 

In [6]:
df.describe("overall").show()

+-------+------------------+
|summary|           overall|
+-------+------------------+
|  count|            296337|
|   mean| 4.393450699710128|
| stddev|0.9869053992908551|
|    min|               1.0|
|    max|               5.0|
+-------+------------------+



<span style="color: blue; font-size:20px; font-weight:bold ">Chia dữ liệu thành các nhãn 0 và 1</span> 

In [7]:
#Bucketize data and create labels 0 if overall rating is in (1.0,2.0), otherwise 1
df1 = df.filter("overall !=3")

splits = [-float("inf"), 4.0, float("inf")]

bucketizer = Bucketizer(splits=splits, inputCol="overall", outputCol="label")

df2= bucketizer.transform(df1)

df2.groupBy("overall","label").count().show()

+-------+-----+------+
|overall|label| count|
+-------+-----+------+
|    2.0|  0.0| 10204|
|    5.0|  1.0|188208|
|    1.0|  0.0|  9045|
|    4.0|  1.0| 64809|
+-------+-----+------+



<span style="color: blue; font-size:20px; font-weight:bold ">Lấy mẫu dữ liệu để tạo tập huấn luyện và kiểm tra</span>

In [8]:
#take sample to create train and test dataset
fractions = {1.0 : .1, 0.0 : 1.0}
df3 = df2.stat.sampleBy("label", fractions, 36)
df3.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|19249|
|  1.0|25224|
+-----+-----+



<span style="color: blue; font-size:20px; font-weight:bold ">Chia dữ liệu thành tập huấn luyện và kiểm tra theo tỷ lệ 80-20%</span>

In [9]:
#Split data as 80-20% Train and Test dataset
splitSeed = 5043
trainingData, testData = df3.randomSplit([0.8, 0.2], splitSeed)

Xây dựng pipeline và mô hình Logistic Regression

<span style="color: blue; font-size:20px; font-weight:bold ">Tokenize văn bản</span>

In [10]:
#Tokenize 
tokenizer = RegexTokenizer(inputCol="text",outputCol="reviewTokensUf",pattern="\\s+|[,.()\"]")

remover = StopWordsRemover(stopWords=StopWordsRemover.loadDefaultStopWords("english"),inputCol="reviewTokensUf",outputCol="reviewTokens")

<span style="color: blue; font-size:20px; font-weight:bold ">Chuyển đổi từ văn bản sang vector đếm từ</span>

In [11]:
#converts word documents to vectors of token counts
cv = CountVectorizer(inputCol="reviewTokens",outputCol="cv",vocabSize=296337)

<span style="color: blue; font-size:20px; font-weight:bold ">Mô hình IDF (Inverse Document Frequency):</span>

In [12]:
#IDF model
idf = IDF(inputCol="cv",outputCol="features")

<span style="color: blue; font-size:20px; font-weight:bold ">Logistic Regression</span>

In [13]:
lr = LogisticRegression(maxIter=100,regParam=0.02,elasticNetParam=0.3)

<span style="color: blue; font-size:20px; font-weight:bold ">Tạo pipeline</span>

In [14]:
#Creates a pipeline
steps =  [tokenizer, remover, cv, idf,lr]
pipeline = Pipeline(stages=steps)

<span style="color: blue; font-size:20px; font-weight:bold ">Huấn luyện mô hình</span>

In [15]:
model = pipeline.fit(trainingData)

24/11/21 02:38:47 WARN DAGScheduler: Broadcasting large task binary with size 1969.4 KiB
24/11/21 02:38:54 WARN DAGScheduler: Broadcasting large task binary with size 1968.6 KiB
24/11/21 02:38:54 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/11/21 02:38:54 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/11/21 02:38:54 WARN DAGScheduler: Broadcasting large task binary with size 1968.6 KiB
24/11/21 02:38:54 WARN DAGScheduler: Broadcasting large task binary with size 1968.6 KiB
24/11/21 02:38:54 WARN DAGScheduler: Broadcasting large task binary with size 1968.6 KiB
24/11/21 02:38:55 WARN DAGScheduler: Broadcasting large task binary with size 1968.6 KiB
24/11/21 02:38:55 WARN DAGScheduler: Broadcasting large task binary with size 1968.6 KiB
24/11/21 02:38:55 WARN DAGScheduler: Broadcasting large task binary with size 1968.6 KiB
24/11/21 02:38:55 WARN DAGScheduler: Broadcasting large task binary with size

<span style="color: blue; font-size:20px; font-weight:bold ">Tạo DataFrame chứa Từ vựng và Trọng số</span>

In [16]:
#collecting all metrics
vocabulary = model.stages[2].vocabulary
weights = model.stages[-1].coefficients.toArray()
weights = [float(weight) for weight in weights]

In [17]:
schema = StructType([StructField('word', StringType()),
                     StructField('weight', FloatType())
                     ])
cdf = spark.createDataFrame(zip(vocabulary, weights), schema)

<span style="color: blue; font-size:20px; font-weight:bold ">Hiển thị Top 10 Từ vựng có Trọng số Cao nhất và Thấp nhất</span>

In [18]:
cdf.orderBy(desc("weight")).show(10)

+---------+----------+
|     word|    weight|
+---------+----------+
|    great| 0.5876225|
|   thoses|  0.325535|
|  perfect|0.32343474|
|     easy| 0.2615016|
|   highly|0.25427502|
|     love|0.23299988|
|excellent|0.22146676|
|     nice|0.21586789|
|     good|0.20862874|
|    works|0.20269535|
+---------+----------+
only showing top 10 rows



In [19]:
cdf.orderBy("weight").show(10)

+-------------+-----------+
|         word|     weight|
+-------------+-----------+
|     returned|-0.38842562|
|         poor|-0.33077022|
|      useless|-0.30299458|
|        waste|-0.27846226|
|        broke|-0.26966578|
|         junk| -0.2493974|
|       return|-0.24831308|
|disappointing|-0.22999014|
|    returning|-0.21706156|
| disappointed|-0.21414408|
+-------------+-----------+
only showing top 10 rows



<span style="color: blue; font-size:20px; font-weight:bold ">Tạo Dự đoán và Đánh giá Mô hình</span>

In [20]:
predictions = model.transform(testData)

In [21]:
evaluator = BinaryClassificationEvaluator()  
areaUnderROC = evaluator.evaluate(predictions)

24/11/21 02:39:18 WARN DAGScheduler: Broadcasting large task binary with size 1986.0 KiB


<span style="color: blue; font-size:20px; font-weight:bold ">Hiển thị Dự đoán</span>

In [22]:
predictions.show()

24/11/21 02:39:22 WARN DAGScheduler: Broadcasting large task binary with size 2003.5 KiB


+----------+-------+--------------------+--------------------+--------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|      asin|overall|          reviewText|             summary|unixReviewTime|                text|label|      reviewTokensUf|        reviewTokens|                  cv|            features|       rawPrediction|         probability|prediction|
+----------+-------+--------------------+--------------------+--------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|7245456313|    1.0|I wish I would ha...|Defective - Be Ca...|    1354492800|Defective - Be Ca...|  0.0|[defective, -, be...|[defective, -, ca...|(71899,[0,11,15,1...|(71899,[0,11,15,1...|[1.99668098749145...|[0.88044816229074...|       0.0|
|7245456313|    5.0|I bought thi

<span style="color: blue; font-size:20px; font-weight:bold ">Đánh giá Mô hình</span>

In [23]:
#model evaluation
lp = predictions.select("label", "prediction")
counttotal = predictions.count()
correct = lp.filter(col("label") == col("prediction")).count()
wrong = lp.filter(~(col("label") == col("prediction"))).count()
ratioWrong = float(wrong) / float(counttotal)
lp = predictions.select(  "prediction","label")
counttotal = float(predictions.count())
correct = lp.filter(col("label") == col("prediction")).count()
wrong = lp.filter("label != prediction").count()
ratioWrong=wrong/counttotal
ratioCorrect=correct/counttotal
trueneg =( lp.filter(col("label") == 0.0).filter(col("label") == col("prediction")).count()) /counttotal
truepos = (lp.filter(col("label") == 1.0).filter(col("label") == col("prediction")).count())/counttotal
falseneg = (lp.filter(col("label") == 0.0).filter(~(col("label") == col("prediction"))).count())/counttotal
falsepos = (lp.filter(col("label") == 1.0).filter(~(col("label") == col("prediction"))).count())/counttotal

precision= truepos / (truepos + falsepos)
recall= truepos / (truepos + falseneg)
#fmeasure= 2  precision  recall / (precision + recall)
accuracy=(truepos + trueneg) / (truepos + trueneg + falsepos + falseneg)

24/11/21 02:39:28 WARN DAGScheduler: Broadcasting large task binary with size 1983.3 KiB
24/11/21 02:39:33 WARN DAGScheduler: Broadcasting large task binary with size 1983.3 KiB
24/11/21 02:39:39 WARN DAGScheduler: Broadcasting large task binary with size 1983.3 KiB
24/11/21 02:39:43 WARN DAGScheduler: Broadcasting large task binary with size 1983.3 KiB
24/11/21 02:39:47 WARN DAGScheduler: Broadcasting large task binary with size 1983.5 KiB
24/11/21 02:39:50 WARN DAGScheduler: Broadcasting large task binary with size 1983.5 KiB
24/11/21 02:39:54 WARN DAGScheduler: Broadcasting large task binary with size 1983.5 KiB
24/11/21 02:39:57 WARN DAGScheduler: Broadcasting large task binary with size 1983.5 KiB


In [24]:
print('counttotal   :', counttotal     )
print('correct      :', correct        )
print('wrong        :', wrong          )
print('ratioWrong   :', ratioWrong     )
print('ratioCorrect :', ratioCorrect   )
print('truen        :', trueneg          )
print('truep        :', truepos          )
print('falsen       :', falseneg         )
print('falsep       :', falsepos         )
print('precision    :', precision      )
print('recall       :', recall         )
#print('fmeasure     :', fmeasure       )
print('accuracy     :', accuracy       )

counttotal   : 9003.0
correct      : 7776
wrong        : 1227
ratioWrong   : 0.13628790403198934
ratioCorrect : 0.8637120959680107
truen        : 0.3361101854937243
truep        : 0.5276019104742864
falsen       : 0.08863712095968011
falsep       : 0.04765078307230923
precision    : 0.9171654759606103
recall       : 0.8561643835616438
accuracy     : 0.8637120959680107


<span style="color: blue; font-size:20px; font-weight:bold ">Hiển thị các dự đoán không tích cực (prediction == 0.0)</span>

In [25]:
predictions.filter(col("prediction") == 0.0)\
.select("summary","reviewTokens","overall","prediction")\
.orderBy(desc("rawPrediction")).show(5)

24/11/21 02:40:00 WARN DAGScheduler: Broadcasting large task binary with size 1996.3 KiB


+--------------------+--------------------+-------+----------+
|             summary|        reviewTokens|overall|prediction|
+--------------------+--------------------+-------+----------+
|Buyer Beware - Yo...|[buyer, beware, -...|    2.0|       0.0|
|Awful Phone and T...|[awful, phone, te...|    1.0|       0.0|
|DO NOT BUY HERE I...|[buy, need, custo...|    1.0|       0.0|
|                JUNK|[junk, well, rece...|    1.0|       0.0|
|Poor 3-9x40 Hamme...|[poor, 3-9x40, ha...|    1.0|       0.0|
+--------------------+--------------------+-------+----------+
only showing top 5 rows



<span style="color: blue; font-size:20px; font-weight:bold ">Hiển thị các dự đoán tích cực (prediction == 1.0)</span>

In [26]:
predictions.filter(col("prediction")== 1.0)\
.select("summary","reviewTokens","overall","prediction")\
.orderBy("rawPrediction").show(5)

24/11/21 02:40:05 WARN DAGScheduler: Broadcasting large task binary with size 1996.2 KiB


+--------------------+--------------------+-------+----------+
|             summary|        reviewTokens|overall|prediction|
+--------------------+--------------------+-------+----------+
|My DROID Story an...|[droid, story, co...|    5.0|       1.0|
| great trucker phone|[great, trucker, ...|    5.0|       1.0|
|    Favorite EDC Bag|[favorite, edc, b...|    4.0|       1.0|
|One of My Favorit...|[one, favorites!!...|    4.0|       1.0|
|Best Hopper I've ...|[best, hopper, us...|    4.0|       1.0|
+--------------------+--------------------+-------+----------+
only showing top 5 rows



<span style="color: blue; font-size:20px; font-weight:bold ">Lưu mô hình đã huấn luyện</span>

In [27]:
dir = "sentiment/"
model.write().overwrite().save(dir)

24/11/21 02:40:11 WARN TaskSetManager: Stage 168 contains a task of very large size (1385 KiB). The maximum recommended task size is 1000 KiB.
24/11/21 02:40:12 WARN TaskSetManager: Stage 171 contains a task of very large size (1153 KiB). The maximum recommended task size is 1000 KiB.


In [28]:
dir = "sentiment/"
model = PipelineModel.load(dir)

<span style="color: blue; font-size:20px; font-weight:bold ">Đọc dữ liệu từ MongoDB và hiển thị schema</span>

In [29]:
df = spark.read.format("mongo").load()
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- prediction: double (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp_ms: string (nullable = true)



<span style="color: blue; font-size:20px; font-weight:bold ">Đọc lại dữ liệu từ MongoDB và chọn cột "timestamp_ms" và "text"</span>

In [30]:
df = spark.read.format("mongo").load().select("timestamp_ms","text")
# Kiểm tra số lượng giá trị null trong cột 'text'
null_count = df.filter(df.text.isNull()).count()
print(f"Số lượng giá trị null trong cột 'text': {null_count}")

Số lượng giá trị null trong cột 'text': 0


In [31]:
# Loại bỏ các hàng có giá trị null trong cột 'text'
df = df.na.drop(subset=["text"])

In [32]:
df.printSchema()

root
 |-- timestamp_ms: string (nullable = true)
 |-- text: string (nullable = true)



<span style="color: blue; font-size:20px; font-weight:bold ">Dùng splits để bucketize dữ liệu nếu cần thiết</span>

In [33]:
splits = [-float("inf"), 0, float("inf")]
#bucketizer = Bucketizer(inputCol="timestamp_ms",outputCol="sentiment",splits=splits)

#df5= bucketizer.transform(df)
predictions = model.transform(df)
result_df = predictions.select('text', 'prediction')

# Chuyển đổi sang Pandas DataFrame
result_pd = result_df.toPandas()


24/11/21 02:40:16 WARN DAGScheduler: Broadcasting large task binary with size 1951.2 KiB


In [34]:
# Định dạng và hiển thị kết quả
styled_result = result_pd.style.set_table_attributes('style="width: 80%; border-collapse: collapse;"') \
                                .set_caption("Dự đoán Sentiment") \
                                .highlight_max(color='lightgreen') \
                                .highlight_min(color='lightcoral') \
                                    .set_properties(**{'text-align': 'left'})


# Hiển thị kết quả
styled_result


,text,prediction
0,Abe Uta fights back tears to cheer for her bro’s gold! 🥲 🇯🇵 JudoTV Judo JudoOlympics Sport Olympics Paris2024,0.000000
1,JudoJuniors 🇹🇯 was absolutely bursting with emotion! Let’s relive that incredible atmosphere once more 🫠 SlowMotionRecap SlowMotion JudoJuniors Dushanbe Tajikistan Judo Sport RoadToLA2028,1.000000
2,JudoJuniors 🇹🇯 was absolutely bursting with emotion! Let’s relive that incredible atmosphere once more 🫠 SlowMotionRecap SlowMotion JudoJuniors Dushanbe Tajikistan Judo Sport RoadToLA2028,1.000000
3,Abe Uta fights back tears to cheer for her bro’s gold! 🥲 🇯🇵 JudoTV Judo JudoOlympics Sport Olympics Paris2024,0.000000
4,JudoJuniors 🇹🇯 was absolutely bursting with emotion! Let’s relive that incredible atmosphere once more 🫠 SlowMotionRecap SlowMotion JudoJuniors Dushanbe Tajikistan Judo Sport RoadToLA2028,1.000000
5,JudoJuniors 🇹🇯 was absolutely bursting with emotion! Let’s relive that incredible atmosphere once more 🫠 SlowMotionRecap SlowMotion JudoJuniors Dushanbe Tajikistan Judo Sport RoadToLA2028,1.000000
6,"The hour numerals and markers, as well as the hands, of Club Sport neomatik 39 are deeply coated with Superluminova and glow blue in the dark.",1.000000
7,"Excited to be in Ouargla, south of Algeria, with ambassador & to celebrate the new school year with sport & play. Our champion gives the 1st PE session. ForEveryChild a champion!",0.000000


<span style="color: red; font-size:30px; font-weight:bold ">Mục đích</span>

### Mô Hình Phân Loại: Dựa trên dự đoán (prediction) đều là 1.000000, mô hình đang thực hiện phân loại nhị phân với tất cả các văn bản đều được dự đoán là tích cực. Điều này cho thấy mô hình có thể là một mô hình phân loại cảm xúc nhị phân

### Dựa trên các kết quả dự đoán, mô hình có thể được áp dụng để phân tích các bài đăng trên mạng xã hội hoặc các bài viết liên quan đến thể thao và hoạt động ngoài trời, nhằm xác định xem các bài viết này mang cảm xúc tích cực hay không.

### File "streamlistener" là một phần quan trọng trong quy trình phân tích cảm xúc tweet theo thời gian thực. Nó kết nối với Kafka để đọc dữ liệu streaming, tiền xử lý dữ liệu, áp dụng mô hình phân tích cảm xúc và lưu trữ kết quả vào MongoDB. Điều này cho phép theo dõi và phân tích cảm xúc của người dùng trên Twitter một cách liên tục và kịp thời.